In [24]:
import rebound
import numpy as np
import random

In [25]:
#draw mass from normal dist assuming Earth/Venus/Mercury density
def get_mass(rp):
    rs, drs_u, drs_l = 1.092, 0.191, 0.109                #radius of sun, upper/lower error bars (solar radii)
    drp_u, drp_l = rp*drs_u/rs, rp*drs_l/rs               #err. prop. for planet radius (assume solar error dominates, earth radii)
    gcm2msrp = 1.3e-7                                     #g/cm^3 -> M_sun/R_Earth^3
    rho = 5.4                                             #avg density of Earth, Venus and Mercury
    return 4./3.*np.pi*(np.random.normal(rp, np.mean([drp_u,drp_l])))**3 *rho*gcm2msrp

#run sim
def sim(sim_id):
    sim = rebound.Simulation()
    sim.integrator="whfast"
    sim.ri_whfast.safe_mode = 0
    sim.G = 4*np.pi**2
    tmax = 1e9
    
    #orb params
    rp1,rp2,rp3 = 0.764, 0.668, 1.11                      #radius of planets (earth radii)
    emax = 0.2
    imax = 0.1
    
    sim.add(m=1.071)
    sim.add(m=get_mass(rp1), a=0.0719, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=get_mass(rp2), a=0.0847, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=get_mass(rp3), a=0.1045, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.move_to_com()
    
    p = sim.particles
    sim.exit_min_distance = p[1].a*(2*p[1].m/3)**(1./3.)  #use smaller hill radius as exit condition
    sim.dt = 2*np.pi*p[1].a**(3./2.) / 50
    
    #stable = [True] # assume it's stable and update if not
    #try:
    #    sim.integrate(tmax)
    #except:
    #    stable = [False]